In [1]:
# ymyD7ijqAGHxBJYOpjrHZtlC2mHZYEN6
# ygqDyviPRT7ZOkLP

from amadeus import Client, ResponseError
import requests
from math import sin, cos, sqrt, atan2, radians
# import pandas as pd
# import numpy as np
import sqlite3
import json
import traceback
from sqlite3 import IntegrityError
import csv
import datetime
import functools
from sqlalchemy import exc

amadeus = Client(
    client_id='ymyD7ijqAGHxBJYOpjrHZtlC2mHZYEN6',
    client_secret='ygqDyviPRT7ZOkLP'
)


In [2]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import create_engine, Column, Integer, String, func, desc, Float, distinct
from sqlalchemy.ext.declarative import declarative_base
# engine = create_engine('sqlite:///:memory:', echo=False)


In [3]:
# Base = automap_base()
Base = declarative_base()
engine = create_engine('sqlite:///event_travel.db')

In [4]:
# Base.prepare(engine, reflect=True)

In [5]:
# for c in Base.classes:
#     print(c)

In [6]:

class MyEvents(Base):
    __tablename__ = 'my_events'
    __table_args__ = {'extend_existing': True}

    id = Column(String, primary_key=True)
    name = Column(String)
    date = Column(String)
    time = Column(String)
    price_low = Column(Float)
    price_high = Column(Float)
    street_address = Column(String)
    city = Column(String)
    state = Column(String)
    lat = Column(Float)
    long = Column(Float)
    distance = Column(Float)
    my_lat = Column(Float)
    my_long = Column(Float)
    my_city = Column(String)
    my_state = Column(String)
    depart_airport = Column(String)
    dest_airport = Column(String)
    
    def __repr__(self):
        return f"<MyEvents(id={self.id}, \
        name={self.name}, \
        date={self.date}, \
        time={self.time}, \
        price_low={self.price_low}, \
        price_high={self.price_high}, \
        street_address={self.street_address}, \
        city={self.city}, \
        state={self.state}, \
        lat={self.lat}, \
        long={self.long}, \
        distance={self.distance}, \
        my_lat={self.my_lat}, \
        my_long={self.my_long}, \
        my_city={self.my_city}, \
        my_state={self.my_state}, \
        depart_airport={self.depart_airport}, \
        dest_airport={self.dest_airport}>"  
    

In [7]:
class Airports(Base):
    __tablename__ = "airports"
    __table_args__ = {'extend_existing': True}
        
    id = Column(Integer, primary_key=True)
    name  = Column(String)
    city = Column(String)
    iata = Column(String)
    lat = Column(String)
    long = Column(String)
    timezone = Column(Float)
    
    def __repr__(self):
        return f"<Airports(id={self.id}, \
        name={self.name}), \
        city={self.city}, \
        iata={self.iata}, \
        lat={self.lat}, \
        long={self.long}, \
        timzone={self.timezone}>"


In [8]:
class RawEvents(Base):
    __tablename__ = "raw_events"
    __table_args__ = {'extend_existing': True}
        
    id = Column(String, primary_key=True)
    name  = Column(String)
    classification = Column(String)
    json = Column(String)
    
    def __repr__(self):
        return f"<RawEvents(id={self.id}, \
        name={self.name}), \
        classification={self.classification}, \
        json={self.json}>"



In [9]:
Base.metadata.create_all(engine)

In [10]:
Session = sessionmaker(bind=engine)
session = Session()

In [11]:
def get_distance(lat1, lon1, lat2, lon2):
    
    # approximate radius of earth in km
    R = 6373.0
    Miles_per_Km = 0.621371

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c * Miles_per_Km

    return distance

In [149]:
def find_closest_airports(lat, long, r):
    print(lat, long)
    ret = []
    best = r

    for airport in session.query(Airports):

        distance = get_distance(lat, long, airport.lat, airport.long)
        if distance <= r:
            data = {'id':airport.id,'name':airport.name,'distance':distance,'airport':airport.iata}
            if distance < best:
                ret.insert(0,data)
                best = distance
            else:
                ret.append(data)
                
    
    return sorted(ret, key=lambda k: k['distance'])             

In [150]:
where_are_you = 'SLC'

In [151]:
likely_departure_points = [
    [40.790152, -111.979038, 'Salt Lake City', 'UT'],
     [36.085044, -115.149928, 'Las Vegas', 'NV'],
     [39.851704, -104.673811, 'Denver', 'CO']
]

for dept in likely_departure_points:
    print(dept)
    ret = find_closest_airports(dept[0],dept[1],30)
    for x in ret:
        print(x)

[40.790152, -111.979038, 'Salt Lake City', 'UT']
40.790152 -111.979038
{'id': 3536, 'name': 'Salt Lake City International Airport', 'distance': 0.13286185225979918, 'airport': 'SLC'}
[36.085044, -115.149928, 'Las Vegas', 'NV']
36.085044 -115.149928
{'id': 3877, 'name': 'McCarran International Airport', 'distance': 0.360713200823457, 'airport': 'LAS'}
[39.851704, -104.673811, 'Denver', 'CO']
39.851704 -104.673811
{'id': 3751, 'name': 'Denver International Airport', 'distance': 0.6920960602071604, 'airport': 'DEN'}


In [15]:
def extract_event_json(event):
    try:
        name = event['name']
        event_id = event['id']

        venue = event['_embedded']['venues'][0]
        location = {'address':venue['address']['line1'],'city':venue['city']['name'],'state':venue['state']['stateCode'],
                'latlong':[float(venue['location']['latitude']),float(venue['location']['longitude'])]}

        latlong = location['latlong']
        airports = find_closest_airports(latlong[0], latlong[1], 20)
        event['airports'] = airports
    except KeyError:
        print('KeyError')
#         traceback.print_exc()
        event = {}

    return event

In [16]:
@functools.lru_cache(maxsize=128)
def request(page_number, classification, size):
    params = {'apikey':'kNkNPU0S8LOzOUrjDWpLto6LEv91lAFM', 'page':page_number, 
                      'countryCode':'US', 'classificationName':classification,'size':size}
    url = f"https://app.ticketmaster.com/discovery/v2/events.json"
    return requests.get(url, params)

In [107]:
@functools.lru_cache(maxsize=128)
def event_search(page_number, keyword, size, end_date):
    params = {'apikey':'kNkNPU0S8LOzOUrjDWpLto6LEv91lAFM', 'page':page_number, 
                      'countryCode':'US', 'keyword':keyword,'size':size, 'endDateTime': end_date}
    url = f"https://app.ticketmaster.com/discovery/v2/events.json"
    return requests.get(url, params)

In [108]:
request.cache_info()

CacheInfo(hits=0, misses=0, maxsize=128, currsize=0)

In [140]:
response = event_search(0, 'eagles', 200, '2020-12-31T00:00:00Z').json()

In [167]:
for event in response.get('_embedded').get('events'):
    name = event.get('name')
    event_id = event.get('id')
    classifications = event.get('classifications')

    segment_name = classifications[0].get('segment').get('name')
    genre = classifications[0].get('genre',{'name':''}).get('name')
    price_range = event.get('priceRanges',[{'min':0,'max':0}])[0]  
    dates = event.get('dates')
    start = dates.get('start').get('localDate')
    venue = event.get('_embedded').get('venues')[0]
    location = venue.get('location')
    city = venue.get('city').get('name')
    state = venue.get('state').get('name')
    latlong = venue.get('location')
    print(latlong)
#     40.790152 -111.979038
    airports = find_closest_airports(float(latlong.get('latitude')),float(latlong.get('longitude')), 500)
    closest_airport = airports[0]
    
    print(f'name:{name}|date:{start}|location:{city}, {state}|segment:{segment_name}|genre:{genre}|price:{price_range.get("min")}-{price_range.get("max")}|airport:{closest_airport.get("airport")}|event_id:{event_id}')
    

{'longitude': '-74.016182', 'latitude': '40.305836'}
40.305836 -74.016182
name:Best Of The Eagles|date:2020-08-21|location:Oceanport, New Jersey|segment:Music|genre:Undefined|price:35.0-59.5|airport:JFK|event_id:k7v1Fp8fgvoPV
{'longitude': '-74.016182', 'latitude': '40.305836'}
40.305836 -74.016182
name:Best Of The Eagles|date:2020-08-20|location:Oceanport, New Jersey|segment:Music|genre:Undefined|price:35.0-59.5|airport:JFK|event_id:k7vGFp26sZvI3
{'longitude': '-96.75646586', 'latitude': '32.77507215'}
32.77507215 -96.75646586
name:97.1 The Eagle Presents Korn & Faith No More|date:2020-08-21|location:Dallas, Texas|segment:Music|genre:Metal|price:29.5-135.0|airport:DFW|event_id:vvG1YZp7M6CMSD
{'longitude': '-121.510309', 'latitude': '39.031072'}
39.031072 -121.510309
name:96.9 The Eagle Welcomes: Sammy Hagar & The Circle and special guests|date:2020-09-18|location:Wheatland, California|segment:Music|genre:Rock|price:29.5-350.0|airport:SMF|event_id:G5vYZpdbBFaGu
{'longitude': '-84.79489

In [24]:
request.cache_info()

CacheInfo(hits=0, misses=0, maxsize=128, currsize=0)

In [18]:
def find_events_by_classification(page_number=0, classification='music'):
    remaining_pages = True
    total_pages = 0
    page_count = 0
    ret = []
    while remaining_pages:
        try:
            r = request(page_number, classification, 25) 
            data = r.json()
            page = data['page']
            size = page['size']
            page_number = page['number']
            total_pages = page['totalPages']
            events = data['_embedded']['events']
            print(f'page: {page_number}/{total_pages}')
            for event in events:
                event_json = extract_event_json(event)
                if len(event_json) > 0:
                    name = event_json['name']
                    if 'priceRanges' in event_json:
            #             print(name)
                        line = (event_json['id'], name, classification, json.dumps(event_json))
                        temp = session.query(RawEvents.id).filter_by(id=line[0]).first()
                        if temp is None:
                            re = RawEvents(id=line[0], name=line[1], classification=line[2], json=line[3])
                            session.add(re)
#                         execute_insert_query('insert into raw_events(event_id, name, classification, json) values(?,?,?,?)', line,cur,conn)
        except KeyError:
            traceback.print_exc()
        except IntegrityError:
            traceback.print_exc()
            session.rollback()

        remaining_pages = ( page_number * size ) < 300
        page_number += 1
        
    session.commit()


 

In [139]:
find_events_by_classification(0,'sports')

page: 0/157
page: 1/157
page: 2/157
page: 3/157
page: 4/157
page: 5/157
page: 6/157
page: 7/157
page: 8/157
page: 9/157
page: 10/157
page: 11/157
page: 12/157


In [256]:

def get_event_classifications():
    for classification in session.query(distinct(RawEvents.classification)):
        print(classification)

In [257]:
get_event_classifications()

('sports',)
('music',)


In [285]:
session.query(distinct(RawEvents.name)) \
.filter(RawEvents.classification=='music') \
.order_by(RawEvents.name).all()

[(' Alanis Morissette w/special guest Garbage & also appearing Liz Phair'),
 ('2020 B105 Country Megaticket'),
 ('2020 Country Megaticket'),
 ('2020 Jiffy Lube Live Country Megaticket'),
 ('2020 Jones Beach Country Megaticket'),
 ('2020 KSON Country Megaticket'),
 ('2020 Pnc Bank Arts Center Country Megaticket Presented By Pennzoil'),
 ('2021 CMA Fest'),
 ('A Thousand Thoughts: A live documentary with the Kronos Quartet'),
 ('Alan Jackson'),
 ('Alanis Morissette w/ special Guest Garbage & also appearing Liz Phair'),
 ('Alanis Morissette w/special guest Garbage & also appearing Liz Phair'),
 ('Alanis Morissette w/special guest Garbage & also sppearing Liz Phair'),
 ('An Evening with Michael Buble in Concert'),
 ('Backstreet Boys: DNA World Tour'),
 ('Barenaked Ladies'),
 ('Billy Joel'),
 ('Billy Joel - In Concert'),
 ('Brad Paisley : Live From The Drive-In'),
 ('Celine Dion: Courage World Tour'),
 ('Cher: Here We Go Again Tour'),
 ("Chris Stapleton's All-American Roadshow"),
 ('Chris St

In [286]:
def get_event_names(classification):
    return session.query(distinct(RawEvents.name)) \
    .filter(RawEvents.classification=='music') \
    .order_by(RawEvents.name).all()
    

In [288]:
get_event_names('music')

[(' Alanis Morissette w/special guest Garbage & also appearing Liz Phair'),
 ('2020 B105 Country Megaticket'),
 ('2020 Country Megaticket'),
 ('2020 Jiffy Lube Live Country Megaticket'),
 ('2020 Jones Beach Country Megaticket'),
 ('2020 KSON Country Megaticket'),
 ('2020 Pnc Bank Arts Center Country Megaticket Presented By Pennzoil'),
 ('2021 CMA Fest'),
 ('A Thousand Thoughts: A live documentary with the Kronos Quartet'),
 ('Alan Jackson'),
 ('Alanis Morissette w/ special Guest Garbage & also appearing Liz Phair'),
 ('Alanis Morissette w/special guest Garbage & also appearing Liz Phair'),
 ('Alanis Morissette w/special guest Garbage & also sppearing Liz Phair'),
 ('An Evening with Michael Buble in Concert'),
 ('Backstreet Boys: DNA World Tour'),
 ('Barenaked Ladies'),
 ('Billy Joel'),
 ('Billy Joel - In Concert'),
 ('Brad Paisley : Live From The Drive-In'),
 ('Celine Dion: Courage World Tour'),
 ('Cher: Here We Go Again Tour'),
 ("Chris Stapleton's All-American Roadshow"),
 ('Chris St

In [291]:
def store_event(event, my_location):  
    try:
        event_id = event['id']
        temp = session.query(MyEvents.id).filter_by(id=event_id).first()
        if temp is not None:
            print(f'skipping {temp[0]}')
        else:
            dates = event['dates']['start']
            print(f'{dates["dateTBD"]} {dates["dateTBA"] } {dates["timeTBA"]}')
            if (not dates["dateTBD"] and not dates["dateTBA"] and not dates["timeTBA"]):
                sales = event['sales']['public']

                prices = event['priceRanges']
                venue = event['_embedded']['venues'][0]
                location = {'address':venue['address']['line1'],'city':venue['city']['name'],'state':venue['state']['stateCode'],
                            'latlong':[float(venue['location']['latitude']),float(venue['location']['longitude'])]}

                event_date = dates['localDate']
                event_time = dates['localTime']
                price_range = [prices[0]['min'],prices[0]['max']]
                distance = get_distance(my_location[0],my_location[1],location['latlong'][0],location['latlong'][1])
                latlong = location['latlong']
                airports = find_closest_airports(latlong[0], latlong[1], 20)


                airport_count = len(airports)
                within_year = datetime.datetime.strptime(event_date,"%Y-%m-%d") < datetime.datetime.today() + datetime.timedelta(days=350)
                print(f"Airports: {airport_count} date: {event_date} within_year: {within_year}")

                if airport_count > 0 and \
                datetime.datetime.strptime(event_date,"%Y-%m-%d") > datetime.datetime.today() and \
                datetime.datetime.strptime(event_date,"%Y-%m-%d") < datetime.datetime.today() + datetime.timedelta(days=350):

                    val = (event_id,
                           event['name'],
                           event_date,
                           event_time,
                           price_range[0],
                           price_range[1],
                           location['address'],
                           location['city'],
                           location['state'],
                           latlong[0],
                           latlong[1],
                           distance,
                           my_location[0],
                           my_location[1],
                           my_location[2],
                           my_location[3],
                           my_location[4],
                           airports[0]['airport']
                          )

#                     print(f"storing {val} at {my_location}")
                    me = MyEvents(id=val[0],
                                  name=val[1],
                                  date=val[2],
                                  time=val[3],
                                  price_low=val[4],
                                  price_high=val[5],
                                  street_address=val[6],
                                  city=val[7],
                                  state=val[8],
                                  lat=val[9],
                                  long=val[10],
                                  distance=val[11],
                                  my_lat=val[12],
                                  my_long=val[13],
                                  my_city=val[14],
                                  my_state=val[15],
                                  depart_airport=val[16],
                                  dest_airport=val[17])
                    session.add(me)
                    session.commit()
    except KeyError:
        session.rollback()
        traceback.print_exc()
        print(event)





In [292]:
def select_events(keyword, my_location):    
    for event in session.query(RawEvents).filter(RawEvents.name.like(f'%{keyword}%')).order_by(RawEvents.name):
        event_json = json.loads(event.json)
        store_event(event_json, my_location)   

In [293]:
select_events('New York', [40.790152, -111.979038, 'Salt Lake City', 'UT', 'SLC'])

skipping 1A4ZAOJGkds8gZB
skipping 1A4ZAOJGkds8AZS
skipping 1A4ZAOJGkdsC4Zm
skipping 1A4ZAOJGkds34vZ
skipping 1A4ZAxkGkdI-Kah
skipping k7vGFp6rPas1j
skipping vv178ZpkGkR-24R0
skipping vv1A7ZAq4GkdkiKZc
skipping vv1A7ZAq4GkdkcSZW
skipping 1ApZAx6GkeySODv
skipping vv1AAZAxbGkdanvY_
False False False
Airports: 0 date: 2020-11-01 within_year: True
skipping vvG1IZpFhIc33Y
skipping vvG1IZpF5JcvgQ
False False True
skipping vvG1VZp6C0btI5
skipping vvG1VZp6C0sate
skipping k7vGFp63-j-4-
skipping k7vGFp63-Ps4R
skipping k7vGFp63-1o4f
skipping k7vGFp63-d-41
skipping k7vGFp63-F-4o
skipping k7vGFp63-bv4_
skipping k7vGFp63-0v4c
skipping k7vGFp63-_s4L
skipping k7vGFp63-ns4s
skipping k7vGFp63-ov4p
skipping k7vGFp6AqeZTK
skipping k7vGFp6Aq7ZT_
skipping k7vGFp6AqeOT4
skipping k7vGFp6AqAOTr
skipping k7vGFp6Aqd8T9
skipping k7vGFp6AqZZT2
skipping k7vGFp6Aqv8To
skipping k7vGFp6AqkYTN
skipping vv17FZp6GkD1--E2
skipping vv17FZp6GkD1x-EK
skipping vvG1HZp6EVAxp_
skipping vvG1HZp6EVkxph
skipping G5viZpF2ma34j


In [251]:
session.rollback()

In [252]:
preferred_event_names = ['Atlanta', 'New York']

likely_departure_points = [
    [40.790152, -111.979038, 'Salt Lake City', 'UT', 'SLC']]
#     [36.085044, -115.149928, 'Las Vegas', 'NV'],
#     [39.851704, -104.673811, 'Denver', 'CO']]

for event_name in preferred_event_names:
    for location in likely_departure_points:
        print(f'Getting {event_name} from {location[2]}')
        select_events(event_name,location)

Getting Atlanta from Salt Lake City
False False False
Airports: 1 date: 2020-10-11 within_year: True
False False False
Airports: 1 date: 2020-09-27 within_year: True
False False False
Airports: 1 date: 2020-08-28 within_year: True
False False False
Airports: 1 date: 2020-11-08 within_year: True
False False False
Airports: 1 date: 2020-11-29 within_year: True
False False False
Airports: 1 date: 2020-08-14 within_year: True
False False False
Airports: 1 date: 2020-12-06 within_year: True
False False False
Airports: 1 date: 2020-09-13 within_year: True
False False False
Airports: 1 date: 2020-12-20 within_year: True
False False False
Airports: 1 date: 2020-08-21 within_year: True
False False False
Airports: 1 date: 2020-10-29 within_year: True
False False False
Airports: 1 date: 2020-10-05 within_year: True
False False False
Airports: 1 date: 2020-09-03 within_year: True
False False False
Airports: 0 date: 2020-12-27 within_year: True
False False False
Airports: 1 date: 2020-12-13 within_

In [248]:
def get_my_events(order_column):
    return session.query(MyEvents).order_by(order_column).all()
#     conn = create_connection()
#     cur = conn.cursor()
#     cur.execute(f'select name, price_low, price_high, distance, date, city, state, depart_airport, dest_airport from my_events {order_by}')
#     out = cur.fetchall()
#     conn.close()
#     return out



In [253]:
get_my_events(None)

[<MyEvents(id=vvG1zZpF2NYJqP,         name=Atlanta Falcons vs. Carolina Panthers,         date=2020-10-11,         time=13:00:00,         price_low=60.0,         price_high=425.0,         street_address=1 AMB Drive NW,         city=Atlanta,         state=GA,         lat=33.755389,         long=-84.400823,         distance=1585.1151916840117,         my_lat=40.790152,         my_long=-111.979038,         my_city=Salt Lake City,         my_state=UT,         depart_airport=SLC,         dest_airport=ATL>,
 <MyEvents(id=vvG1zZpF2NskqM,         name=Atlanta Falcons vs. Chicago Bears,         date=2020-09-27,         time=13:00:00,         price_low=110.0,         price_high=455.0,         street_address=1 AMB Drive NW,         city=Atlanta,         state=GA,         lat=33.755389,         long=-84.400823,         distance=1585.1151916840117,         my_lat=40.790152,         my_long=-111.979038,         my_city=Salt Lake City,         my_state=UT,         depart_airport=SLC,         dest_air

In [297]:
def get_flight_search_json(origin, destination, event_date):

    d = datetime.datetime.strptime(event_date,"%Y-%m-%d")
    delta = datetime.timedelta(days=1)
    start = datetime.datetime.strftime(d - delta, "%Y-%m-%d")
    end = datetime.datetime.strftime(d + delta, "%Y-%m-%d")
    
    return {
              "currencyCode": "USD",
              "originDestinations": [
                {
                  "id": "1",
                  "originLocationCode": origin,
                  "destinationLocationCode": destination,
                  "departureDateTimeRange": {
                    "date": start,
                    "time": "8:00:00"
                  }
                },
                {
                  "id": "2",
                  "originLocationCode": destination,
                  "destinationLocationCode": origin,
                  "departureDateTimeRange": {
                    "date": end,
                    "time": "17:00:00"
                  }
                }
              ],
              "travelers": [
                {
                  "id": "1",
                  "travelerType": "ADULT"
                }
              ],
              "sources": [
                "GDS"
              ],
              "searchCriteria": {
                "maxFlightOffers": 1

              }
            }

In [298]:
@functools.lru_cache(maxsize=128)
def search_flights(origin, destination, event_date):
    print(f"Getting flights from {origin} to {destination} on {event_date}")

    body = get_flight_search_json(origin, destination, event_date)
    response = amadeus.shopping.flight_offers_search.post(body)
    return response.data

In [299]:
flight_results = search_flights('SLC', 'SFO', '2021-05-01')

Getting flights from SLC to SFO on 2021-05-01


In [300]:
flight_results

[{'type': 'flight-offer',
  'id': '1',
  'source': 'GDS',
  'instantTicketingRequired': False,
  'nonHomogeneous': False,
  'oneWay': False,
  'lastTicketingDate': '2020-07-17',
  'numberOfBookableSeats': 7,
  'itineraries': [{'duration': 'PT2H2M',
    'segments': [{'departure': {'iataCode': 'SLC',
       'terminal': '2',
       'at': '2021-04-30T15:00:00'},
      'arrival': {'iataCode': 'SFO',
       'terminal': '1',
       'at': '2021-04-30T16:02:00'},
      'carrierCode': 'DL',
      'number': '2021',
      'aircraft': {'code': '221'},
      'operating': {'carrierCode': 'DL'},
      'duration': 'PT2H2M',
      'id': '1',
      'numberOfStops': 0,
      'blacklistedInEU': False}]},
   {'duration': 'PT1H46M',
    'segments': [{'departure': {'iataCode': 'SFO',
       'terminal': '1',
       'at': '2021-05-02T18:50:00'},
      'arrival': {'iataCode': 'SLC',
       'terminal': '2',
       'at': '2021-05-02T21:36:00'},
      'carrierCode': 'DL',
      'number': '2102',
      'aircraft': {

In [42]:
t_flight = flight_results[0]['itineraries'][0]['segments'][0]['departure']
r_flight = flight_results[0]['itineraries'][1]['segments'][0]['departure']
print(f"depart:{t_flight['iataCode']} at:{t_flight['at']} ret:{r_flight['iataCode']} at:{r_flight['at']} ${flight_results[0]['price']['grandTotal']}")

depart:SLC at:2021-04-30T15:00:00 ret:SFO at:2021-05-02T18:50:00 $236.20


In [56]:
my_events = get_my_events("order by distance, price_low, date")
with open('my_events.csv', mode='w') as events_file:
    event_writer = csv.writer(events_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    current_event = ''
    for event in my_events:
        response = get_flights(event[7], event[8], event[4])
        if len(response) == 0:
            print(f"Can't find this flight: {event[7]} {event[8]} {event[4]}")
        else:
            flight_results = response[0]
            t_flight = flight_results['itineraries'][0]['segments'][0]['departure']
            r_flight = flight_results['itineraries'][1]['segments'][0]['departure']
            low = event[1] + float(flight_results['price']['grandTotal'])
            high = event[2] + float(flight_results['price']['grandTotal'])
            if current_event != event[0]:
                print(event[0])
                current_event=event[0]
            print(f"total cost:${low:.2f} to ${high:.2f} event tickets:${event[1]:.2f} to ${event[2]:.2f} flight:${flight_results['price']['grandTotal']} distance:{event[3]:.2f} date:{event[4]}, city:{event[5]}, state:{event[6]}, depart:{t_flight['iataCode']} at:{t_flight['at']} ret:{r_flight['iataCode']} at:{r_flight['at']}")
            event_writer.writerow(event)
        

Getting flights from SLC to LAS on 2020-08-21
Shania Twain "Let's Go!" - The Las Vegas Residency
total cost:$176.20 to $367.20 event tickets:$60.00 to $251.00 flight:$116.20 distance:366.71 date:2020-08-21, city:Las Vegas, state:NV, depart:SLC at:2020-08-20T20:58:00 ret:LAS at:2020-08-22T17:25:00
Getting flights from SLC to LAS on 2020-08-22
total cost:$307.20 to $498.20 event tickets:$60.00 to $251.00 flight:$247.20 distance:366.71 date:2020-08-22, city:Las Vegas, state:NV, depart:SLC at:2020-08-21T20:58:00 ret:LAS at:2020-08-23T19:00:00
Getting flights from SLC to LAS on 2020-08-26
total cost:$196.20 to $387.20 event tickets:$60.00 to $251.00 flight:$136.20 distance:366.71 date:2020-08-26, city:Las Vegas, state:NV, depart:SLC at:2020-08-25T17:30:00 ret:LAS at:2020-08-27T19:00:00
Getting flights from SLC to LAS on 2020-08-28
total cost:$176.20 to $367.20 event tickets:$60.00 to $251.00 flight:$116.20 distance:366.71 date:2020-08-28, city:Las Vegas, state:NV, depart:SLC at:2020-08-27T2

In [301]:
def get_flights(origin, destination, event_date):
    flights_json = search_flights(origin, destination, event_date)
    if len(flights_json) == 0:
        return {}
        print(f"Can't find this flight: {origin} {destination} {event_date}")
    else:
        flight_results = flights_json[0]
        t_flight = flight_results['itineraries'][0]['segments'][0]['departure']
        r_flight = flight_results['itineraries'][1]['segments'][0]['departure']
        cost = float(flight_results['price']['grandTotal'])
        return {"cost":cost,"t_fight":t_flight, "r_flight":r_flight}
        

In [303]:
get_flights('SLC', 'SFO', '2021-05-01')

{'cost': 212.2,
 't_fight': {'iataCode': 'SLC', 'terminal': '2', 'at': '2021-04-30T15:00:00'},
 'r_flight': {'iataCode': 'SFO', 'terminal': '1', 'at': '2021-05-02T18:50:00'}}

In [33]:
get_flights.cache_info()

CacheInfo(hits=14, misses=18, maxsize=128, currsize=15)

In [3]:
@functools.lru_cache(maxsize=128)
def find_hotel(city, date):
    hotel_list = amadeus.shopping.hotel_offers.get(cityCode=city)
    print(hotel_list)
    first_hotel = hotel_list.result.get('data')[0].get('hotel').get('hotelId')
    response = amadeus.shopping.hotel_offers_by_hotel.get(hotelId=first_hotel, checkInDate=date)
    
    return response

In [4]:
@functools.lru_cache(maxsize=128)
def find_hotel2(lat,long,radius,date):
    hotel_list = amadeus.shopping.hotel_offers.get(latitude=lat, longitude=long, radius=radius, radiusUnit='MILE')
    first_hotel = hotel_list.result.get('data')[0].get('hotel').get('hotelId')
    response = amadeus.shopping.hotel_offers_by_hotel.get(hotelId=first_hotel, checkInDate=date)
    
    return response

In [5]:
# find_hotel2(36.085044, 115.149928, 20, '2021-05-01')

In [6]:
r = find_hotel('BOI', '2021-05-01')

In [7]:
r.result

{'data': {'type': 'hotel-offers',
  'hotel': {'type': 'hotel',
   'hotelId': 'HIBOI093',
   'chainCode': 'HI',
   'dupeId': '700035441',
   'name': 'Holiday Inn Exp University Area',
   'rating': '2',
   'cityCode': 'BOI',
   'latitude': 43.59787,
   'longitude': -116.18655,
   'address': {'lines': ['475 W. PARKCENTER BLVD.'],
    'postalCode': '83706',
    'cityName': 'BOISE',
    'countryCode': 'US',
    'stateCode': 'ID'},
   'contact': {'phone': '+1 208 3452002', 'fax': '+1 208 3458354'},
   'description': {'lang': 'en',
    'text': 'The Holiday Inn Express Boise University hotel is the latest in contemporary design and modern styling. Conveniently located within the Boise Downtown area and Boise State University area and a short distance from I 84 as well as I 184 we are only minutes from the Boise Airport as well as a variety of restaurants shopping and entertainment. Guests will appreciate being within walking distance to Boise State University  BSU  Bronco Stadium and the Green

In [429]:
r = amadeus.shopping.hotel_offers.get(latitude=36.085044, longitude=115.149928, radius=200, radiusUnit='MILE')

In [430]:
r.result

{'data': [],
 'meta': {'links': {'next': 'https://test.api.amadeus.com/v2/shopping/hotel-offers?latitude=36.085044&longitude=115.149928&radius=200&radiusUnit=MILE&page[offset]=UTEG6LJFL2RO_100'}}}

In [8]:
R = amadeus.shopping.hotel_offers.get(cityCode='BOI')

In [9]:
R.result

{'data': [{'type': 'hotel-offers',
   'hotel': {'type': 'hotel',
    'hotelId': 'HIBOI093',
    'chainCode': 'HI',
    'dupeId': '700035441',
    'name': 'Holiday Inn Exp University Area',
    'rating': '2',
    'cityCode': 'BOI',
    'latitude': 43.59787,
    'longitude': -116.18655,
    'hotelDistance': {'distance': 1.4, 'distanceUnit': 'KM'},
    'address': {'lines': ['475 W. PARKCENTER BLVD.'],
     'postalCode': '83706',
     'cityName': 'BOISE',
     'countryCode': 'US',
     'stateCode': 'ID'},
    'contact': {'phone': '+1 208 3452002', 'fax': '+1 208 3458354'},
    'description': {'lang': 'en',
     'text': 'The Holiday Inn Express Boise University hotel is the latest in contemporary design and modern styling. Conveniently located within the Boise Downtown area and Boise State University area and a short distance from I 84 as well as I 184 we are only minutes from the Boise Airport as well as a variety of restaurants shopping and entertainment. Guests will appreciate being with